In [1]:
!pip3 install ultralytics

  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.5/913.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 6.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 6.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 5.9 MB/s eta 0:00:0000:0100:01
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 7.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.2
    Uninstalling numpy-2.2.2:
      Successfully uninstalled numpy-2.2.2


In [5]:
conda install yaml

Channels:
 - conda-forge
Platform: osx-arm64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")
    
torch.backends.mps.is_built() 
torch.backends.mps.is_available()
device = torch.device("mps")

tensor([1.], device='mps:0')


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 경로 설정
dataset_path = '/Users/lywoo/Desktop/꽃 인식 DNN/datasets'  # 원본 데이터 디렉토리
images_path = os.path.join(dataset_path, '/Users/lywoo/Desktop/꽃 인식 DNN/datasets/images')  # 이미지 파일 디렉토리
labels_path = os.path.join(dataset_path, '/Users/lywoo/Desktop/꽃 인식 DNN/datasets/labels')  # 라벨 파일 디렉토리

train_path = os.path.join(dataset_path, "train")  # 학습 데이터 저장 디렉토리
val_path = os.path.join(dataset_path, "val")      # 검증 데이터 저장 디렉토리
test_path = os.path.join(dataset_path, "test")    # 테스트 데이터 저장 디렉토리

# 디렉토리 생성
os.makedirs(os.path.join(train_path, "images"), exist_ok=True)
os.makedirs(os.path.join(train_path, "labels"), exist_ok=True)
os.makedirs(os.path.join(val_path, "images"), exist_ok=True)
os.makedirs(os.path.join(val_path, "labels"), exist_ok=True)
os.makedirs(os.path.join(test_path, "images"), exist_ok=True)
os.makedirs(os.path.join(test_path, "labels"), exist_ok=True)

# 이미지와 라벨 파일 리스트 가져오기
image_files = sorted([f for f in os.listdir(images_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
label_files = sorted([f for f in os.listdir(labels_path) if f.endswith('.txt')])

print(len(image_files), len(label_files))

# for label_file in label_files:
#     new_label = label_file.replace('15.3', '')
#     os.rename(os.path.join(labels_path, label_file), os.path.join(labels_path, new_label))

# 이미지와 라벨의 이름이 동일한지 확인
assert len(image_files) == len(label_files), "이미지와 라벨 파일 개수가 다릅니다."
for img, lbl in zip(image_files, label_files):
    assert os.path.splitext(img)[0] == os.path.splitext(lbl)[0], f"파일 이름이 다릅니다: {img}, {lbl}"

# 데이터를 6:2:2로 나누기
train_images, temp_images, train_labels, temp_labels = train_test_split(
    image_files, label_files, test_size=0.4, random_state=42  # 40%를 val+test로 분리
)
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.5, random_state=42  # 나머지 40% 중 절반씩 분리
)

# 파일 복사 함수
def copy_files(file_list, src_folder, dest_folder):
    for file_name in file_list:
        src = os.path.join(src_folder, file_name)
        dest = os.path.join(dest_folder, file_name)
        shutil.copy(src, dest)

# 이미지와 라벨 복사
copy_files(train_images, images_path, os.path.join(train_path, "images"))
copy_files(train_labels, labels_path, os.path.join(train_path, "labels"))

copy_files(val_images, images_path, os.path.join(val_path, "images"))
copy_files(val_labels, labels_path, os.path.join(val_path, "labels"))

copy_files(test_images, images_path, os.path.join(test_path, "images"))
copy_files(test_labels, labels_path, os.path.join(test_path, "labels"))

# 데이터 분할 결과 출력
print(f"Train 데이터: {len(train_images)}개")
print(f"Validation 데이터: {len(val_images)}개")
print(f"Test 데이터: {len(test_images)}개")

162 162
Train 데이터: 97개
Validation 데이터: 32개
Test 데이터: 33개


In [10]:
import yaml

data = {
    'train' : '/Users/lywoo/Desktop/꽃 인식 DNN/datasets/train',
    'val' : '/Users/lywoo/Desktop/꽃 인식 DNN/datasets/val',
    'test' : '/Users/lywoo/Desktop/꽃 인식 DNN/datasets/test',
    'names' : {0 : 'flower'},
    'nc' : 1
}

with open('./flower.yaml', 'w') as f:
    yaml.dump(data, f)

with open('./flower.yaml', 'r') as f:
    lines = yaml.safe_load(f)
    print(lines)

{'names': {0: 'flower'}, 'nc': 1, 'test': '/Users/lywoo/Desktop/꽃 인식 DNN/datasets/test', 'train': '/Users/lywoo/Desktop/꽃 인식 DNN/datasets/train', 'val': '/Users/lywoo/Desktop/꽃 인식 DNN/datasets/val'}


In [5]:
from ultralytics import YOLO
import torch

model = YOLO('yolo11n.pt').to(device)
model.train(data='/Users/lywoo/Desktop/꽃 인식 DNN/flower.yaml', epochs=50)

engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/Users/lywoo/Desktop/꽃 인식 DNN/flower.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, op

train: Scanning /Users/lywoo/Desktop/꽃 인식 DNN/datasets/train/labels... 97 images, 0 backgrounds, 0 corrupt: 100%|██████████| 97/97 [00:01<00:00, 69.07it/s] 

train: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/train/images/KakaoTalk_20240418_130907386_01.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/train/images/KakaoTalk_20240418_130907386_05.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/train/images/KakaoTalk_20240418_130907386_08.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/train/images/KakaoTalk_20240418_130907386_09.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/train/images/KakaoTalk_20240418_130907386_12.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/train/images/KakaoTalk_20240418_130907386_13.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/train/images/KakaoTalk_20240418_130907386_16.jpg: corrupt JPEG rest


val: Scanning /Users/lywoo/Desktop/꽃 인식 DNN/datasets/val/labels... 32 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32/32 [00:00<00:00, 61.75it/s]

val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/val/images/KakaoTalk_20240418_130907386_02.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/val/images/KakaoTalk_20240418_130907386_03.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/val/images/KakaoTalk_20240418_130907386_04.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/val/images/KakaoTalk_20240418_130907386_11.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/val/images/KakaoTalk_20240418_130907386_15.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/val/images/KakaoTalk_20240418_131003449_10.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/val/images/KakaoTalk_20240418_131432057_05.jpg: corrupt JPEG restored and saved
val: WARNING 

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:14<00:00, 14.89s/it]


                   all         32         85      0.881      0.106      0.111     0.0771
Speed: 1.5ms preprocess, 91.8ms inference, 0.0ms loss, 133.4ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x323a49a00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

## **예측(txt값 뽑아내기)**

In [9]:
from ultralytics import YOLO

model = YOLO('/Users/lywoo/Desktop/꽃 인식 DNN/runs/detect/train2/weights/best.pt').to(device)

test_data_path = '/Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images'

results = model.predict(source=test_data_path, save=True, save_txt=True)
print(results)


image 1/33 /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/20240412_131359.jpg: 384x640 1 flower, 7300.3ms
image 2/33 /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/20240412_131412.jpg: 384x640 2 flowers, 60.1ms
image 3/33 /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/20240412_132408.jpg: 384x640 4 flowers, 21.5ms
image 4/33 /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/20240412_132412.jpg: 384x640 6 flowers, 20.3ms
image 5/33 /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/20240412_132451.jpg: 384x640 5 flowers, 24.8ms
image 6/33 /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/20240412_132453.jpg: 384x640 1 flower, 21.4ms
image 7/33 /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/20240412_132458.jpg: 384x640 3 flowers, 28.8ms
WARNING ⚠️ NMS time limit 2.050s exceeded
image 8/33 /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/20240412_132507.jpg: 384x640 9 flowers, 23.0ms
image 9/33 /Users/lywoo/Desktop/꽃 ᄋ

## **뽑아낸 값과 정답 label 비교**

In [14]:
from ultralytics import YOLO
import warnings

warnings.filterwarnings('ignore')

# 모델 불러오기
model = YOLO('/Users/lywoo/Desktop/꽃 인식 DNN/runs/detect/train2/weights/best.pt').to(device)

# 테스트 데이터 평가
metrics = model.val(data='/Users/lywoo/Desktop/꽃 인식 DNN/flower.yaml', split='test')

# 평가 결과 출력 (Precision, Recall, mAP 등)
print(f"Precision: {metrics.box.map50:.4f}")
print(f"Recall: {metrics.box.recall:.4f}")
print(f"mAP@50: {metrics.box.map50:.4f}")
print(f"mAP@50-95: {metrics.box.map:.4f}")

YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/labels.cache... 33 images, 0 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]

val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/KakaoTalk_20240418_130907386.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/KakaoTalk_20240418_130907386_06.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/KakaoTalk_20240418_130907386_07.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/KakaoTalk_20240418_130907386_10.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/KakaoTalk_20240418_130907386_14.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/KakaoTalk_20240418_131003449_01.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/lywoo/Desktop/꽃 인식 DNN/datasets/test/images/KakaoTalk_20240418_131003449_13.jpg: corrupt JPEG restored and saved
val: WARN


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/3 [00:00<?, ?it/s]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|███▎      | 1/3 [00:04<00:09,  4.78s/it]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:11<00:00,  3.95s/it]


                   all         33         80      0.809      0.212      0.251      0.164
Speed: 2.5ms preprocess, 16.8ms inference, 0.0ms loss, 189.5ms postprocess per image
Results saved to runs/detect/val4
Precision: 0.2507


AttributeError: 'Metric' object has no attribute 'recall'. See valid attributes below.

    Class for computing evaluation metrics for YOLOv8 model.

    Attributes:
        p (list): Precision for each class. Shape: (nc,).
        r (list): Recall for each class. Shape: (nc,).
        f1 (list): F1 score for each class. Shape: (nc,).
        all_ap (list): AP scores for all classes and all IoU thresholds. Shape: (nc, 10).
        ap_class_index (list): Index of class for each AP score. Shape: (nc,).
        nc (int): Number of classes.

    Methods:
        ap50(): AP at IoU threshold of 0.5 for all classes. Returns: List of AP scores. Shape: (nc,) or [].
        ap(): AP at IoU thresholds from 0.5 to 0.95 for all classes. Returns: List of AP scores. Shape: (nc,) or [].
        mp(): Mean precision of all classes. Returns: Float.
        mr(): Mean recall of all classes. Returns: Float.
        map50(): Mean AP at IoU threshold of 0.5 for all classes. Returns: Float.
        map75(): 